## 1. Protect them from themselves

In [1]:
import pandas as pd
import glob
import os

In [5]:
data_folder = "datasets"
def load_crss_table(table_name, years=[2022, 2023]):
    dfs = []

    for year in years:
        folder = os.path.join(data_folder, f"CRSS{year}CSV")
        filepath = os.path.join(folder, f"{table_name}.csv")

        if not os.path.exists(filepath):
            print(f"⚠️ File not found: {filepath}")
            continue

        print(f"Loading: {filepath}")
        df = pd.read_csv(filepath)
        df["year"] = year
        dfs.append(df)

    if not dfs:
        raise ValueError("No files found — check folder names and CSV file names.")
    
    return pd.concat(dfs, ignore_index=True)

In [6]:
accidents = load_crss_table("accident")
vehicles  = load_crss_table("vehicle")
persons   = load_crss_table("person")

print("Accidents:", accidents.shape)
print("Vehicles:", vehicles.shape)
print("Persons:", persons.shape)


Loading: datasets/CRSS2022CSV/accident.csv
Loading: datasets/CRSS2023CSV/accident.csv
Loading: datasets/CRSS2022CSV/vehicle.csv


/var/folders/31/gfwq7d5x3dxf1qzfgh1tvksr0000gn/T/ipykernel_33345/1065187931.py:14: DtypeWarning: Columns (55,57,69,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


Loading: datasets/CRSS2023CSV/vehicle.csv


/var/folders/31/gfwq7d5x3dxf1qzfgh1tvksr0000gn/T/ipykernel_33345/1065187931.py:14: DtypeWarning: Columns (57,59,69,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


Loading: datasets/CRSS2022CSV/person.csv
Loading: datasets/CRSS2023CSV/person.csv
Accidents: (104058, 81)
Vehicles: (182217, 170)
Persons: (254563, 113)


In [8]:
pd.set_option('display.max_columns', None) 
persons.head()

,CASENUM,VE_FORMS,VEH_NO,PER_NO,PSU,PSU_VAR,PSUSTRAT,REGION,REGIONNAME,URBANICITY,URBANICITYNAME,STRATUM,STRATUMNAME,PJ,WEIGHT,MONTH,MONTHNAME,HOUR,HOURNAME,MINUTE,MINUTENAME,HARM_EV,HARM_EVNAME,MAN_COLL,MAN_COLLNAME,SCH_BUS,SCH_BUSNAME,MOD_YEAR,MOD_YEARNAME,VPICMAKE,VPICMAKENAME,VPICMODEL,VPICMODELNAME,VPICBODYCLASS,VPICBODYCLASSNAME,MAKE,MAKENAME,BODY_TYP,BODY_TYPNAME,ICFINALBODY,ICFINALBODYNAME,GVWR_FROM,GVWR_FROMNAME,GVWR_TO,GVWR_TONAME,TOW_VEH,TOW_VEHNAME,AGE,AGENAME,AGE_IM,AGE_IMNAME,SEX,SEXNAME,SEX_IM,SEX_IMNAME,PER_TYP,PER_TYPNAME,DEVTYPE,DEVTYPENAME,DEVMOTOR,DEVMOTORNAME,INJ_SEV,INJ_SEVNAME,INJSEV_IM,INJSEV_IMNAME,SEAT_POS,SEAT_POSNAME,SEAT_IM,SEAT_IMNAME,REST_USE,REST_USENAME,REST_MIS,REST_MISNAME,HELM_USE,HELM_USENAME,HELM_MIS,HELM_MISNAME,AIR_BAG,AIR_BAGNAME,EJECTION,EJECTIONNAME,EJECT_IM,EJECT_IMNAME,DRINKING,DRINKINGNAME,PERALCH_IM,PERALCH_IMNAME,ALC_STATUS,ALC_STATUSNAME,ATST_TYP,ATST_TYPNAME,ALC_RES,ALC_RESNAME,DRUGS,DRUGSNAME,HOSPITAL,HOSPITALNAME,STR_VEH,LOCATION,LOCATIONNAME,SPEC_USE,SPEC_USENAME,EMER_USE,EMER_USENAME,ROLLOVER,ROLLOVERNAME,IMPACT1,IMPACT1NAME,FIRE_EXP,FIRE_EXPNAME,MAK_MOD,MAK_MODNAME,year
0,202203729314,2,1,1,75,75,19,3,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",2,Rural Area,5,Stratum 5 - NLMY PV Serious Injury,4144,29.239053,1,January,9,9:00am-9:59am,48,48,12,Motor Vehicle In-Transport,2,Front-to-Front,0,No,1988.0,1988,449.0,Mercedes-Benz,14014.0,300,13.0,Sedan/Saloon,42.0,Mercedes-Benz,4.0,"4-door sedan, hardtop",0.0,Not Applicable,11.0,"Class 1: 6,000 lbs. or less (2,722 kg or less)",12.0,"Class 2: 6,001 - 10,000 lbs. (2,722 - 4,536 kg)",0.0,No Trailers,93,93 Years,93,93 Years,1,Male,1,Male,1,Driver of a Motor Vehicle In-Transport,NaN,NaN,NaN,NaN,4,Fatal Injury (K),4,Fatal Injury (K),11,"Front Seat, Left Side",11,"Front Seat, Left Side",20,None Used/Not Applicable,7,None Used/Not Applicable,20,Not Applicable,7,None Used/Not Applicable,20,Not Deployed,1,Totally Ejected,1,Totally Ejected,8,Not Reported,0,No (Alcohol Not Involved),0,Test Not Given,0,Test Not Given,996,Test Not Given,8,Not Reported,5,EMS Ground,0,0,Occupant of a Motor Vehicle,0.0,No Special Use Noted,0.0,Not Applicable,0.0,No Rollover,12.0,12 Clock Point,0.0,No or Not Reported,42031.0,Mercedes-Benz 200/220/230/240/ 250/260/280/300...,2022
1,202203729314,2,2,1,75,75,19,3,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",2,Rural Area,5,Stratum 5 - NLMY PV Serious Injury,4144,29.239053,1,January,9,9:00am-9:59am,48,48,12,Motor Vehicle In-Transport,2,Front-to-Front,0,No,2002.0,2002,478.0,Nissan,1911.0,Pathfinder,7.0,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,35.0,Nissan/Datsun,14.0,"Compact Utility (Utility Vehicle Categories ""S...",0.0,Not Applicable,11.0,"Class 1: 6,000 lbs. or less (2,722 kg or less)",11.0,"Class 1: 6,000 lbs. or less (2,722 kg or less)",0.0,No Trailers,29,29 Years,29,29 Years,1,Male,1,Male,1,Driver of a Motor Vehicle In-Transport,NaN,NaN,NaN,NaN,2,Suspected Minor Injury (B),2,Suspected Minor Injury (B),11,"Front Seat, Left Side",11,"Front Seat, Left Side",3,Shoulder and Lap Belt Used,0,No Indication of Misuse,20,Not Applicable,7,None Used/Not Applicable,1,Deployed- Front,0,Not Ejected,0,Not Ejected,0,No (Alcohol Not Involved),0,No (Alcohol Not Involved),0,Test Not Given,0,Test Not Given,996,Test Not Given,0,No (drugs not involved),5,EMS Ground,0,0,Occupant of a Motor Vehicle,0.0,No Special Use Noted,0.0,Not Applicable,0.0,No Rollover,12.0,12 Clock Point,0.0,No or Not Reported,35401.0,Nissan/Datsun Pathfinder,2022
2,202203729974,1,1,1,48,48,12,3,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",1,Urban Area,6,Stratum 6 - LMY PV Minor Injury,85,48.398719,1,January,21,9:00pm-9:59pm,38,38,42,Tree (Standing Only),0,The First Harmful Event was Not a Collision wi...,0,No,2020.0,2020,478.0,Nissan,1913.0,Rogue,8.0,Crossover Utility Vehicle (CUV),35.0,Nissan/Datsun,14.0,"Compact Utility (Utility Vehicle Categories ""S...",0.0,Not Applicable,11.0,"Class 1: 6,000 lbs. or less (2,722 kg or less)",11.0,

## 2. Main solution (Altron data)

### Data Preprocessing and Cleaning

In [23]:
import pandas as pd
try:
    excel_url = "https://adbdatathon.s3.af-south-1.amazonaws.com/Datathon/Vehicle_Data.xlsx"
    
    print(f"Attempting to read Excel file from URL: {excel_url}\n")
    
    # No special options needed for a direct public HTTPS link
    df_excel = pd.read_excel(excel_url)
    
    print("✅ Successfully read Excel file from URL!")
    print("First 5 rows of the Excel data:")
    print(df_excel.head())

except Exception as e:
    print(f"❌ Error reading Excel file: {e}")
try:
    excel_url = "https://adbdatathon.s3.af-south-1.amazonaws.com/Datathon/part-00590-tid-6661262080813627668-5b71b33c-8263-4f63-9ea0-b9e2e435cab4-130881-1.c000.snappy.parquet"
    
    print(f"Attempting to read Excel file from URL: {excel_url}\n")
    
    # No special options needed for a direct public HTTPS link
    df_parquet = pd.read_parquet(excel_url)
    
    print("✅ Successfully read Excel file from URL!")
    print("First 5 rows of the Excel data:")
    print(df_parquet.head())

except Exception as e:
    print(f"❌ Error reading parquet file: {e}")


Attempting to read Excel file from URL: https://adbdatathon.s3.af-south-1.amazonaws.com/Datathon/Vehicle_Data.xlsx

✅ Successfully read Excel file from URL!
First 5 rows of the Excel data:
              IMEI        Make             Model           Colour    Year  \
0  860305051123794  VOLKSWAGEN     VW 27X - POLO      WHITE / WIT  2018.0   
1  860305051124123     HYUNDAI              ATOS  SILVER / SILWER  2022.0   
2  860305051147801        FORD            RANGER      GREY / GRYS  2022.0   
3  860305051168823      SUZUKI            ERTIGA      WHITE / WIT  2023.0   
4  860305051198176  VOLKSWAGEN  VW 240-POLO VIVO  SILVER / SILWER  2011.0   

    Age  Gender  
0  38.0    Male  
1  46.0    Male  
2  45.0    Male  
3  64.0    Male  
4  40.0  Female  
Attempting to read Excel file from URL: https://adbdatathon.s3.af-south-1.amazonaws.com/Datathon/part-00590-tid-6661262080813627668-5b71b33c-8263-4f63-9ea0-b9e2e435cab4-130881-1.c000.snappy.parquet

✅ Successfully read Excel file from URL!


In [24]:
df_excel.head()

,IMEI,Make,Model,Colour,Year,Age,Gender
0,860305051123794,VOLKSWAGEN,VW 27X - POLO,WHITE / WIT,2018.0,38.0,Male
1,860305051124123,HYUNDAI,ATOS,SILVER / SILWER,2022.0,46.0,Male
2,860305051147801,FORD,RANGER,GREY / GRYS,2022.0,45.0,Male
3,860305051168823,SUZUKI,ERTIGA,WHITE / WIT,2023.0,64.0,Male
4,860305051198176,VOLKSWAGEN,VW 240-POLO VIVO,SILVER / SILWER,2011.0,40.0,Female


In [25]:
pd.set_option('display.max_columns',500)
df_parquet.head()

,sourceimei,battery_voltage_value,hardware_attached_gps_not_present,unitdatetime,eventid,latitude,longitude,altitude,speed,heading,bearing,odometer,gforce_forward,gforce_backward,gforce_up,gforce_down,gforce_left,gforce_right,engine_hours,classification,closest_suburb,condition,country,distance_suburb,municipality,postal_code,province,road,road_distance_m,rough_road,roadspeed,suburb,town
0,869716067485269,12.484994,1,2025-10-08 11:24:34 PM,5E,-26.122560,28.078880,1639.0,0,80,80,261911,0,50,0,950,0,0,159382,70,None,1,South Africa,0,City of Johannesburg,2090,Gauteng,"""""",0,0,0,Bramley,Johannesburg
1,869716067485269,12.845138,1,2025-10-08 03:59:49 PM,02,-26.122560,28.078880,1639.0,0,80,80,261911,0,50,0,950,0,0,159381,70,None,1,South Africa,0,City of Johannesburg,2090,Gauteng,"""""",0,0,0,Bramley,Johannesburg
2,869716067485269,12.845138,1,2025-10-08 03:59:50 PM,02,-26.122560,28.078880,1639.0,0,80,80,261911,0,50,0,950,0,0,159381,70,None,1,South Africa,0,City of Johannesburg,2090,Gauteng,"""""",0,0,0,Bramley,Johannesburg
3,869716067485269,12.845138,1,2025-10-08 03:59:51 PM,02,-26.122564,28.078883,1639.0,0,80,80,261911,0,50,0,950,0,0,159382,70,None,1,South Africa,0,City of Johannesburg,2090,Gauteng,"""""",0,0,0,Bramley,Johannesburg
4,869716067485269,12.845138,1,2025-10-08 03:59:51 PM,00,-26.122560,28.078880,1639.0,0,80,80,261911,0,50,0,950,0,0,159382,70,None,1,South Africa,0,City of Johannesburg,2090,Gauteng,"""""",0,0,0,Bramley,Johannesburg


In [26]:
df_excel.shape

(949, 7)

This dataset contains information on car makes and models and has 949 rows and 7 columns. 

In [27]:
df_parquet.shape

(58426, 33)

This dataset containing car telemetrics has 58 426 rows and 33 columns. 